In [80]:
import pandas as pd
import datetime
import time
import re
import collections
import warnings
import sklearn
import pickle
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import KFold,train_test_split



In [81]:
# load the data
movies_MD_df = pd.read_csv("data\movies_metadata.csv")
credits_df = pd.read_csv("data\credits.csv")
ratings_df = pd.read_csv(r"data\\ratings.csv")

C:\Users\oferinbar\AppData\Local\Temp\ipykernel_14792\2354645000.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_MD_df = pd.read_csv("data\movies_metadata.csv")


In [82]:
# drop unnecessary columns
movies_MD_df = movies_MD_df.drop(
    columns=[
        "adult",
        "belongs_to_collection",
        "homepage",
        "poster_path",
        "production_companies",
        "production_countries",
        "spoken_languages",
        "status",
        "tagline",
        "video",
        "budget",
        "revenue",
        "vote_average",
        "popularity",
    ]
)
# Remove rows which have NaN values
movies_MD_df = movies_MD_df.dropna()
credits_df = credits_df.dropna()
# change the id data type to int
movies_MD_df["id"] = movies_MD_df["id"].astype(int)
# merge on the id column
movies_creds_df = movies_MD_df.merge(credits_df, on="id")
# print how many items we have
print("Number of items: ", len(movies_creds_df))

movieVector= {"actors": 2000, "directors": 500, "screenPlay":500, "writers": 500}


Number of items:  44488


In [83]:
# data preparation inspired by https://www.kaggle.com/code/surenj/movielens-eda
df= movies_creds_df.copy(deep=True)
df["release_date"] = (pd.to_datetime(df["release_date"], yearfirst= True )  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
genreRegex = re.compile(r": '(.*?)'")
df["genres"] = df["genres"].apply(lambda x: ', '.join(genreRegex.findall(x)))
castRegex = re.compile(r"'name': '(.*?)'")
df["cast"] = df["cast"].apply(lambda x: ', '.join(castRegex.findall(x)))
directorRegex = re.compile(r"'Director', 'name': '(.*?)'")
df["director"] = df["crew"].apply(lambda x: ', '.join(directorRegex.findall(x)))
# writingRegex= re.compile(r"'writing', 'name': '(.*?)'")
# df["writing"] = df["crew"].apply(lambda x: ', '.join(writingRegex.findall(x)))
ScreenplayRegex= re.compile(r"'Screenplay', 'name': '(.*?)'")
df["Screenplay"] = df["crew"].apply(lambda x: ', '.join(ScreenplayRegex.findall(x)))
producerRegex= re.compile(r"'Producer', 'name': '(.*?)'")
df["producer"] = df["crew"].apply(lambda x: ', '.join(producerRegex.findall(x)))
genreCols= df.genres.str.get_dummies(sep=', ') #Creating dummy variables for genres
df = pd.concat([df, genreCols], axis=1)
dfCols2Keep= ['id', 'imdb_id', 'original_title',
       'overview', 'release_date', 'runtime', 'title', 'vote_count', 'cast',
       'crew', 'director',  'Screenplay', 'producer', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']
df.head()

,genres,id,imdb_id,original_language,original_title,overview,release_date,runtime,title,vote_count,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,"Animation, Comedy, Family",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",815011200,81.0,Toy Story,5415.0,...,0,0,0,0,0,0,0,0,0,0
1,"Adventure, Fantasy, Family",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,818985600,104.0,Jumanji,2413.0,...,0,0,0,0,0,0,0,0,0,0
2,"Romance, Comedy",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,819590400,101.0,Grumpier Old Men,92.0,...,0,0,0,0,1,0,0,0,0,0
3,"Comedy, Drama, Romance",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",819590400,127.0,Waiting to Exhale,34.0,...,0,0,0,0,1,0,0,0,0,0
4,Comedy,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,792374400,106.0,Father of the Bride Part II,173.0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# make a colun for each director- only keep the top x directors
topDirectors = df["director"].dropna().value_counts()[:movieVector["directors"]].index.to_list()
directorCols= pd.get_dummies(df["director"])
directorCols= directorCols.loc[::,topDirectors].copy()
print(f'shape of directorCols: {directorCols.shape}')
directorCols.head(1)

shape of directorCols: (44488, 500)


,,John Ford,Michael Curtiz,Alfred Hitchcock,Werner Herzog,Georges Méliès,Woody Allen,Sidney Lumet,Charlie Chaplin,Henry Hathaway,...,Harold S. Bucquet,John Waters,Christopher Nolan,John Brahm,Philippe Garrel,Peter Medak,Tsai Ming-liang,Edward Sedgwick,Byron Haskin,John Madden
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# make a colun for each cast member- only keep the top 15 cast members per movie
cast= df["cast"].apply(lambda x: x.split(", ")[:15]).to_list()
castFlatList= [item for sublist in cast for item in sublist]
# keep only the top x cast members
topCast = pd.Series(castFlatList).value_counts()[:movieVector["actors"]].index.to_list()
castCols= df["cast"].apply(lambda x: x.split(", ")[:15] )
castCols= castCols.apply(pd.Series).stack()
#  only keep rows with top cast members
castCols= castCols[castCols.isin(topCast)]
castCols= pd.get_dummies(castCols.apply(pd.Series).stack()).sum(level=0)
print(f'shape of castCols: {castCols.shape}')
castCols.head(1)


shape of castCols: (27642, 2000)


,,Aamir Khan,Aaron Eckhart,Abhishek Bachchan,Abigail Breslin,Adam Baldwin,Adam Brody,Adam Goldberg,Adam Sandler,Adam Scott,...,Yasiin Bey,Yolande Moreau,Yuen Biao,Yul Brynner,Yuriy Kutsenko,Yves Montand,Yvonne De Carlo,Zach Galifianakis,Zoe Saldana,Zooey Deschanel
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# make a colun for each Screenplay- only keep the top 5 screenplay per movie
Screenplay= df["Screenplay"].apply(lambda x: x.split(", ")[:5]).to_list()
ScreenplayFlatList= [item for sublist in Screenplay for item in sublist]
# keep only the top 200 screenplay
topScreenplay = pd.Series(ScreenplayFlatList).value_counts()[:movieVector["screenPlay"]].index.to_list()
ScreenplayCols= df["Screenplay"].apply(lambda x: x.split(", ")[:5] )
ScreenplayCols= ScreenplayCols.apply(pd.Series).stack()
#  only keep rows with top screenplay
ScreenplayCols= ScreenplayCols[ScreenplayCols.isin(topScreenplay)]
ScreenplayCols= pd.get_dummies(ScreenplayCols.apply(pd.Series).stack()).sum(level=0)
print(f'shape of ScreenplayCols: {ScreenplayCols.shape}')
ScreenplayCols.head(1)


shape of ScreenplayCols: (33312, 500)


,,Abem Finkel,Aben Kandel,Adriano Bolzoni,Agenore Incrocci,Aki Kaurismäki,Akira Kurosawa,Akiva Goldsman,Alan Jay Lerner,Alan Le May,...,William Nicholson,William Roberts,William Rose,Willis Goldbeck,Wilson Mizner,Wim Wenders,Winston Miller,Wolf Mankowitz,Woody Allen,Æneas MacKenzie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
path2Keep= r"data2Keep\\" # path to save the data to
movies_df= pd.concat([df, directorCols, castCols, ScreenplayCols], axis=1).drop(columns= ['cast', 'crew', 'director', 'Screenplay', 'producer'])
print(f'shape of dfMovies: {movies_df.shape}')
stacked= movies_df.iloc[40705:40706].stack()
stacked[stacked!=0]
#  pickle the movies_df 
movies_df.to_pickle(path2Keep + "movies_df.pkl")

shape of dfMovies: (44488, 3030)


In [88]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [89]:
# filter sparse movies, inspired by https://www.kaggle.com/code/morrisb/how-to-recommend-anything-deep-recommender
min_movie_ratings = 1000
filtered_movies = (ratings_df.movieId.value_counts()>min_movie_ratings)
filtered_movies = filtered_movies[filtered_movies].index.tolist()

# Filter sparse users
min_user_ratings = 50
filtered_users = (ratings_df.userId.value_counts()>min_user_ratings)
filtered_users = filtered_users[filtered_users].index.tolist()

# Actual filtering
ratings_df2= ratings_df[(ratings_df.movieId.isin(filtered_movies)) & (ratings_df.userId.isin(filtered_users))]
print('Shape User-Ratings unfiltered:\t{}'.format(ratings_df.shape))
print('Shape User-Ratings filtered:\t{}'.format(ratings_df2.shape))
# dump ratings_df2 to file



Shape User-Ratings unfiltered:	(26024289, 4)
Shape User-Ratings filtered:	(20251813, 4)


In [90]:
# randomly asign filtered_users to test and train sets
train_users, test_users = train_test_split(ratings_df2.userId.value_counts().index, test_size=0.2, random_state=42)
print(f'shapes of train_users, test_users: {train_users.shape, test_users.shape}')
#  dump train_users and test_users to disk
with open(path2Keep +'train_users.pkl', 'wb') as f:
    pickle.dump(train_users, f)
with open(path2Keep +'test_users.pkl', 'wb') as f:
    pickle.dump(test_users, f)
# randomly asign filtered_movies to test and train sets    
train_movies, test_movies = train_test_split(ratings_df2.movieId.value_counts().index, test_size=0.2, random_state=42)
#  dump train_movies and test_movies to disk
print(f'shapes of train_movies, test_movies: {train_movies.shape, test_movies.shape}')
with open(path2Keep +'train_movies.pkl', 'wb') as f:
    pickle.dump(train_movies, f)
with open(path2Keep +'test_movies.pkl', 'wb') as f:
    pickle.dump(test_movies, f)

shapes of train_users, test_users: ((81903,), (20476,))
shapes of train_movies, test_movies: ((2995,), (749,))


In [91]:
#  get the last rating for each user in the test set
lastX= 2
lastXRatingsIndex= ratings_df2.groupby('userId')['timestamp'].nlargest(lastX)
#  get the ratings from ratings_df2 that are in lastXRatings
lastXRatings= ratings_df2[ratings_df2.index.isin(lastXRatingsIndex.index.get_level_values(1))]
# last2Ratings.head()

In [92]:
# sort last2Ratings by userId and timestamp
lastXRatings= lastXRatings.sort_values(by=['userId', 'timestamp'])
print("last 5 ratings of user #4 : ")
print(ratings_df2[ratings_df2.userId == 4].sort_values(by='timestamp', ascending=False).head(5))
print("derived last 2 ratings of user #4:")
print(lastXRatings[lastXRatings.userId == 4].sort_values(by='timestamp', ascending=False))
#  get all ratings from ratings_df2 that are not in the lastXRatings
beforelastXRatings= ratings_df2[~ratings_df2.index.isin(lastXRatingsIndex.index.get_level_values(1))]
print("3 items before the last 2: ")
print(beforelastXRatings[beforelastXRatings.userId == 4].sort_values(by='timestamp', ascending=False).head(3))
print(f'shape of beforelastXRatings: {beforelastXRatings.shape}')
print(f'shape of lastXRatings: {lastXRatings.shape}')

last 5 ratings of user #4 : 
     userId  movieId  rating   timestamp
76        4     2338     2.0  1042674886
66        4     1422     4.0  1042674861
70        4     1644     3.0  1042674845
117       4     4238     4.0  1042674819
61        4      648     4.0  1042674800
derived last 2 ratings of user #4:
    userId  movieId  rating   timestamp
76       4     2338     2.0  1042674886
66       4     1422     4.0  1042674861
3 items before the last 2: 
     userId  movieId  rating   timestamp
70        4     1644     3.0  1042674845
117       4     4238     4.0  1042674819
61        4      648     4.0  1042674800
shape of beforelastXRatings: (20047055, 4)
shape of lastXRatings: (204758, 4)


In [93]:
#  pickle the last2Ratings and beforelast2Ratings to path2keep
with open(path2Keep +'lastXRatings.pkl', 'wb') as f:
    pickle.dump(lastXRatings, f)
with open(path2Keep +'beforelastXRatings.pkl', 'wb') as f:
    pickle.dump(beforelastXRatings, f)

In [94]:
# ratings_df2.to_csv(path2Keep + 'filtered_ratings_df.csv', index=False)